<a href="https://colab.research.google.com/github/rotom303/Final_Project/blob/main/PreProcessingFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finger Flexion Final Project
Developed by

Alexander Byrd, Aakash Jajoo, Chaoyi Cheng



# Project Setup

In [1]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.io import loadmat, savemat
from scipy.fft import fft, fftfreq
import random
from google.colab import drive

from google.colab import auth
import gspread
from google.auth import default

!pip install deepdiff
from deepdiff import DeepDiff


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 1.9 MB/s eta 0:00:00


**File Directory:**


In [2]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Brain_Computer_Interfaces/Final_Project/

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

proj_data = loadmat('raw_training_data.mat')
leaderboard_data = loadmat('leaderboard_data.mat')
fs = 1000 # Sampling frequency of the signals
numPatients = 3 # This is just to increase reusability.

Mounted at /content/drive
/content/drive/MyDrive/Brain_Computer_Interfaces/Final_Project


# Filter Design


In [3]:
"""
Filter parameters:

fc_passband: a list in form [f1,f2]. f1 and f2 are the corner frequencies of
  a bandpass filter (-3dB attenuation at f1 & f2). Frequencies in between f1 and 
  f2 are kept (called the pass band) while others outside that range are 
  attenuated (called the rejection band). Units = Hz 

order: The order of the bandpass filter used. Increasing the order of a filter
  makes the transition between the pass band and the rejection band sharper. In
  the case of a Butterworth filter, increasing order means the pass band stays
  flat closer to f1 and f2. Increasing the order too much will reduce filter 
  stability and can result in ripples in the pass band or other unpredictable
  behaviour. 4th to 6th Order filters tend to be the best compromise.  

applyNotch: a Boolean for whether or not to apply a notch filter. A Notch filter
  Removes noise at precise frequencies. It is useful for removing artifacts
  from power supplies, which are usually at harmonics of 60Hz.   

f_notch: Selects the frequencies which are removed with a Notch Filter. It must
  be a list, even if it just has 1 element. Units = Hz.

Q: is the quality factor of the notch filter. A low Q will also attenuate
  frequencies near f_notch. A high Q will make the notch filter more precise, 
  but it will not attenuate f_notch as much. 
"""
fc_passband = [75,115]
order = 4
applyNotch = True 
f_notch = [60,120]; 
Q = 50

def apply_filter(raw_signal):
  """
  Input: 
    raw_signal (samples x channels): the raw signal
  Output: 
    clean_data (samples x channels): the filtered signal
  """
  number_of_channels = np.shape(raw_signal)[1] #number of channels
  filteredData = np.zeros(np.shape(raw_signal)); #filtered data output

  # Bandpass Butterworth filter 
  sos = sig.butter(order, fc_passband, 'bandpass', analog=False, fs=fs, output='sos'); # returns filter coefficients
  b_notch = []; a_notch = []
  for f_remove in f_notch:
    b, a = sig.iirnotch(f_remove,Q,fs=fs)
    b_notch.append(b); a_notch.append(a)
  #for each channel
  for chanInd in np.arange(number_of_channels):
    # subtract mean from each datapoint
    currFilt = raw_signal[:, chanInd] - np.mean(raw_signal[:, chanInd]);
    if(applyNotch): 
      for i in range(len(b_notch)):
        currFilt = sig.filtfilt(b_notch[i],a_notch[i],currFilt)
    currFilt = sig.sosfiltfilt(sos, currFilt) # forward-backward digital filter using cascaded second-order sections                                        
    filteredData[:, chanInd] = currFilt
  return filteredData

# Feature Extraction

## Feature Class
Every feature is a child of this class. This class streamlines the process of adding features and how we take extract them from the ECoG data. We can easily change whether a feature is normalized (using standardization method) and whether it uses the raw or filtered ECoG data. 
This class will save the mean and standard deviation of the training data so it can easily standardize both training and testing data.

We can define every feature we consider here, but the only features that are extracted and used in the models will be the ones we instantiate as objects. Every object we instantiate is automatically added to the global list *featFns*, which keeps track of the features we are analyzing. 

In [4]:
global featFns;
featFns = [] # A list that stores all the Feature Objects used
featDict = dict() # a dict that stores each Feature Object. 
  # The dict keys are from its get_name method.

class Feature():
  def __init__(self,isFiltered=True, doNormalize=True):
    self.isFiltered = isFiltered # Whether or not the feature is extracted from the raw or filtered data.
    self.doNormalize = doNormalize # Whether or not to normalize this feature
    self.mean = 0 # mean
    self.std = 1 # standard deviation
    featFns.append(self)

  def __call__(self, signal_data):
    return signal_data

  def get_name(self,nameAddon=None):
    name = type(self).__name__
    if nameAddon is not None:
      name += nameAddon
    name_filter = 'Filtered'
    name_norm = 'Normalized'
    if not self.isFiltered: name_filter = 'not' + name_filter
    if not self.doNormalize: name_norm = 'not' + name_norm
    fullname = name + '_' + name_filter + '_' + name_norm
    featDict[fullname] = self
    return fullname

  def standardize_training(self,training_feat):
    if(self.doNormalize):
      self.mean = np.mean(training_feat)
      self.std = np.std(training_feat)
    return (training_feat - self.mean)/self.std

  def standardize_testing(self,testing_feat):
    return (testing_feat-self.mean) / self.std
  


## Feature Definitions
This is where the functions for the different features are defined. The functions themselves should be written in the __ call__(self, window) method.

Note that the functions are applied to one window in one channel at a time. 

In [5]:
class LineLength(Feature):
  def __call__(self,x):
    return np.sum(np.absolute(np.ediff1d(x)))

class Area(Feature):
  def __call__(self,x):
    return np.sum(np.absolute(x))

class Energy(Feature):
  def __call__(self,x):
    return np.sum(np.square(x))

class ZeroCrossings(Feature):
  def __call__(self,x):
    return np.size(np.nonzero(np.ediff1d(np.sign(x-np.mean(x)))))

class Mean(Feature):
  def __call__(self,x):
    return np.mean(x)

class FreqBand(Feature):
  def __init__(self,f_low,f_high,isFiltered=False,doNormalize=True):
    Feature.__init__(self,isFiltered,doNormalize)
    self.f_low = f_low
    self.f_high = f_high
  
  def __call__(self,signal):
    freq_response = fft(signal)
    N = len(freq_response)
    n = np.arange(N)
    T = N/fs #sampling rate=1000
    freq = n/T 
    power_spectrum = np.abs(freq_response)
    # Find values in frequency vector corresponding to input band
    index_band = np.logical_and(freq >= self.f_low, freq <= self.f_high)
    #average frequency domain magnitude
    avg_mag = np.mean(power_spectrum[index_band])
    return avg_mag

  def get_name(self):
    nameAddon = str(self.f_low) + 'to' + str(self.f_high)
    return Feature.get_name(self,nameAddon)

## Getting Windowed Feats

In [6]:
def NumWins(x,winLen,winDisp,fs=1000):
  """
    Calculates the number of possible full windows that can fit in x
    Inputs:
      x is the signal in the time domain. 
      fs is the sampling frequency of x. Hz.
      winLen is the length of windows. sec
      winDisp is the displacement between the start of each window. sec
  """
  x_duration = len(x)/fs # seconds.
  windows_fit = (x_duration - winLen + winDisp) / (winDisp)
  # default behaviour of int() is to floor float, so using round()
  return round(windows_fit)

def get_features(raw_window,filtered_window):
  """
    Input: 
      raw_window (window_samples x channels): the window of the unfiltered ecog signal 
      filtered_window (window_samples x channels): the window of the filtered ecog signal 
      

    Global Inputs: must be defined outside of the function
      featFns: a list containing the methods to apply as feats. 
    
    Output:s
      features (channels x num_features): the features calculated on each channel for the window
  """
  [window_samples,num_channels]=np.shape(raw_window)
  features=np.empty((num_channels,len(featFns)))
  i = 0
  for feat in featFns:
    if feat.isFiltered: window = filtered_window
    else: window = raw_window
    for chn in range(num_channels):
      current_window = window[:,chn]
      features[chn,i] = feat(current_window)
    i+=1
  return features

def get_windowed_feats(ecog_data, window_length, window_overlap):
  """
    Inputs:
      raw_eeg (samples x channels): the raw signal
      window_length: the window's length
      window_overlap: the window's overlap
    Output: 
      all_feats (num_windows x (channels x features)): the features for each channel for each time window
        note that this is a 2D array. 
  """
  [num_samples,num_channels]=np.shape(ecog_data)
  num_windows = NumWins(ecog_data, window_length,window_overlap, fs) 
  filtered_ecog = apply_filter(ecog_data)
  #convert everything to units of samples
  wLen=round(window_length*fs) #window length in samples
  wDisp=round(window_overlap*fs) #window displacement in samples
  data_feats = np.zeros((num_windows,num_channels*len(featFns))); # stores the features of the window
   
   
  rightmost = num_samples
  for i in range(num_windows):
    raw_window = ecog_data[rightmost-wLen:rightmost,:]
    filtered_window = filtered_ecog[rightmost-wLen:rightmost,:]
    data_feats[-1-i,:] = (get_features(raw_window,filtered_window).flatten())
    rightmost = rightmost - wDisp
  return data_feats


## Feature Normalization
We are using standardization to normalize every feature.
:

In [7]:

def standardize_training(feature_matrix):
  [windows_trn,feats_trn] = np.shape(feature_matrix)
  normFeats = np.empty((windows_trn,feats_trn))
  for i in range(feats_trn):
      normFeats[:,i] = featFns[i].standardize_training(feature_matrix)
  return normFeats

def standardize_testing(feature_matrix):
  [windows_trn,feats_trn] = np.shape(feature_matrix)
  normFeats = np.empty((windows_trn,feats_trn))
  for i in range(feats_trn):
      normFeats[:,i] = featFns[i].standardize_testing(feature_matrix)
  return normFeats

def standardize_both(train_feats, test_feats):
  return standardize_training(train_feats), standardize_testing(test_feats)

## Response Matrix
We will be using a response matrix as the input to each of our learning algorithms. This is because it allows us to associate one window with the *N_wind* windows before it instead of treating each window as an independent case.   

In [8]:
def create_R_matrix(features, N_wind):
  """ 
  Input:
    features (samples (number of windows in the signal) x channels x features): 
      the features you calculated using get_windowed_feats
    N_wind: number of windows to use in the R matrix

  Output:
    R (samples x (N_wind*channels*features))
  """
  features_appended = np.copy(features)
  for i in list(range(N_wind-2, -1, -1)):
      a = features[i]
      features_appended = np.vstack([a, features_appended])
  samples = len(features)   # number of rows = number of windows

  R = np.zeros((samples, 1+(N_wind*len(features[0,:]))))  # len(features[0,:]) = (num of features)*(num of channels)
  lst = np.array(list(range(1, 1+N_wind)))
  R[:, 0] = 1
  
  
  for i in range(len(features[0,:])):   # goes thru each column of the features matrix
    for j in range(len(lst)):
        x = lst[j]
        R[:, x] = features_appended[j : (len(features_appended)-(N_wind-1-j)), i]
    lst = lst + N_wind
  return R


# Define Parameters

## User-Defined Parameters
Below are the user defined parameters. All the parameters are defined here, except for the ones that have already been defined for the filter in the Filter Design section.

In [9]:
# Name of the file to write or read the data from for parameters and R matrices
filename = 'previousRun.pkl'
use_file_params = False 
# If use_file_params is True, then it will overwrite the parameters defined here

# Window Parameters
window_length = 100e-3  # seconds
window_displacement = 50e-3 #seconds
N_winds = 3 # Number of previous windows considered in Response matrix. 

# These Booleans are to make code clearer
uses_raw = False; uses_filtered = True 
not_normalized = False; normalized = True; 
# Different Features analyzed
line_length = LineLength(uses_filtered,normalized)
area = Area(uses_filtered,normalized)
energy = Energy(uses_filtered,normalized)
zero_crossings = ZeroCrossings(uses_filtered,normalized)
mean = Mean(uses_filtered,normalized)
freq_band_5_to_15 = FreqBand(5,15,uses_raw,normalized)
freq_band_20_to_25 = FreqBand(20,25,uses_raw,normalized)
freq_band_75_to_115 = FreqBand(75,115,uses_raw,normalized)
freq_band_125_to_160 = FreqBand(125,160,uses_raw,normalized)
freq_band_160_to_175 = FreqBand(160,175,uses_raw,normalized)

# Training / Testing Split
training_fraction = 2/3 # What fraction of the samples


## Parameter Dictionary
Here we put all the parameters into a dictionary. This makes it easy to export them and compare them between trials. This dictionary will be compared to parameter dictionaries from previous trials, and if they use the same parameters, we will just load the previous feature and R matrices instead of calculating them all again. The saving and loading code is towards the end of the Preparing Data section.

In [10]:
global param_dict;
param_dict = dict()
# Filter Params
param_dict['fs'] = fs
param_dict['fc_passband'] = fc_passband 
param_dict['order'] = order
param_dict['applyNotch'] = applyNotch
param_dict['f_notch'] = f_notch
param_dict['Q'] = Q
# Window Parameters
param_dict['winLen'] = window_length
param_dict['winDisp'] = window_displacement
param_dict['N_winds'] = N_winds
# Feature Parameters
param_dict['featFns'] = []
for feat in featFns:
  param_dict['featFns'].append(feat.get_name())
# Training / Testing Split Parameter
param_dict['training_fraction'] = training_fraction


# Data Preprocessing
Putting the data in a form that can be easily used for different learning algorithms. 

## Helpful Functions

In [11]:
def split_data(data, train_fraction = training_fraction):
  """
    Inputs:
    data = a samples x channels array of data for one patient
    training_fraction = a number between 0 and 1 that represents the fraction of
      data that will be put into the training split. The remaining will be put
      into the testing split. 
    Returns:
      training data, testing data
  """
  m = len(data[:,0]) # Number of samples per channel
  m_training = round(train_fraction*m)
  training_data = data[0:m_training,:]
  testing_data = data[m_training:-1,:]
  return training_data, testing_data

def save_feature_parameters(filename):  
  with open(filename, 'wb') as f:
    # Note that this will overwrite any data already in filename
    pickle.dump(param_dict,f)
  print(f"The parameters and R matrices have been saved in {filename}")

def load_feature_parameters(filename):
  with open(filename, 'rb') as f:
    ref_dict = pickle.load(f)
  return ref_dict

def compare_dicts(dict1,dict2):
  """
    Input: dict1 and dict2 are borth dictionaries. The expectation is that they
      have the same keys. 
    Returns: the number of differences between the keys in dict1 and the keys in
      dict2. It will also print them out.  
  """
  differences = DeepDiff(dict1,dict2)
  num_differences = len(differences.to_dict())
  if num_differences > 0:
    print(f'{filename} uses different parameters from the ones currently set.')
    print(differences)
  return num_differences

## Load Data

This is where all the R Matrices are either loaded or created depending on whether the filename provided has them already. If it does not, then the newly created R matrices will be saved into it. 

In [12]:
raw_ecog = proj_data['train_ecog'][:,0]
raw_glove = proj_data['train_dg'][:,0]
raw_leaderboard = leaderboard_data['leaderboard_ecog'][:,0]


In [13]:
newDataNeeded = True 
# new Data Needed is True by default, but set to False if data matching the
# parameters described in param_dict are found in filename. 

try: # This Try-Except block is in case filename does not exist
  ref_dict = load_feature_parameters(filename)
  # featwinds_total = ref_dict.pop('featwinds_total',None)
  # featwinds_training = ref_dict.pop('featwinds_training',None)
  # featwinds_testing = ref_dict.pop('featwinds_testing',None)
  # featwinds_leaderboard = ref_dict.pop('featwinds_leaderboard',None)
  R_total = ref_dict.pop('R_total',None)
  R_train = ref_dict.pop('R_train',None)
  R_test = ref_dict.pop('R_test',None)
  R_leaderboard = ref_dict.pop('R_leaderboard',None) 
  if compare_dicts(param_dict, ref_dict) == 0:
    newDataNeeded = False
    print(f"{filename} uses the same parameters as currently set, so it will be loaded")
  elif use_file_params:
    newDataNeeded = False
    print(f"{filename} uses different parameters, but use_file_params is True")
except: 
  print(f'{filename} was not found. Creating new a dataset instead')
finally:
  if newDataNeeded:
    # featwinds_total = []; featwinds_leaderboard = []
    # featwinds_training = []; featwinds_testing = []
    #glove_flexion_total = []
    R_total = []; R_leaderboard = []; R_train = []; R_test = []
    for p in range(numPatients):
      feat_matrix_p = get_windowed_feats(raw_ecog[p], window_length, window_displacement)
      R_total.append(create_R_matrix(feat_matrix_p,N_winds))
      
      leaderboard_feats = get_windowed_feats(raw_leaderboard[p], window_length, window_displacement)
      R_leaderboard.append(create_R_matrix(leaderboard_feats,N_winds))

      trn_matrix_p, tst_matrix_p = split_data(feat_matrix_p)
      R_train.append(create_R_matrix(trn_matrix_p,N_winds))
      R_test.append(create_R_matrix(tst_matrix_p,N_winds))

      # glove_flexion_total.append(downsample(raw_glove[p]))
      # featwinds_total.append(feat_matrix_p)
      # featwinds_training.append(trn_matrix_p); 
      # featwinds_testing.append(tst_matrix_p)
      # featwinds_leaderboard.append(leaderboard_feats)
    
    # param_dict['featwinds_total'] = featwinds_total
    # param_dict['featwinds_training'] = featwinds_training
    # param_dict['featwinds_testing'] = featwinds_testing
    # param_dict['featwinds_leaderboard'] = featwinds_leaderboard    
    param_dict['R_total'] = R_total
    param_dict['R_train'] = R_train
    param_dict['R_test'] = R_test
    param_dict['R_leaderboard'] = R_leaderboard
    save_feature_parameters(filename)


previousRun.pkl uses the same parameters as currently set, so it will be loaded


# Split Training Data

# Learning Algorithms